In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock-ts-fa225f46565756e7b0567441810f232f.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "code-mock-staged-v1"

# create thread
thread = await client.threads.create()

print(thread)

In [2]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [3]:
from agent_graph.code_mock_staged_v1.graph import create_compiled_graph
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# graph = create_graph().compile()
graph = create_compiled_graph()
config = { "configurable": {"thread_id": "1", "session_id1": "abc"} }

In [4]:
async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        "event": "reminder",
        # "trigger": True,
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        # "event": "user_message"
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='Hi', id='2ca845d0-2987-4ccb-b977-1b85c5d27a33')], 'event': 'reminder', 'completed_steps': []})
('values', {'messages': [HumanMessage(content='Hi', id='2ca845d0-2987-4ccb-b977-1b85c5d27a33'), HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='1f2f3caa-381d-4bd1-9348-04e6c6efc792')], 'event': 'reminder', 'initialized': True, 'current_stage_idx': 0, 'events': [EventDescriptor(name=<CodingEventType.CODE_EDITOR_CONTENT_CHANGED: 'code_editor_content_changed'>, description='The candidate has changed the code in the editor (shown in diff format).'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_SET_UPDATED: 'user_defined_test_cases_changed'>, description='The candidate has updated the user-defined test cases set.'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_EXECUTED: 'user_defined_test_case_executed'>, description='The candidate has executed the user-defined test cases.

In [5]:
from langchain_core.load.load import load, loads
from langchain_core.load.dump import dumpd, dumps
from langgraph.types import StateSnapshot

state = graph.get_state(config=config)
state

StateSnapshot(values={'messages': [HumanMessage(content='Hi', id='2ca845d0-2987-4ccb-b977-1b85c5d27a33'), HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='1f2f3caa-381d-4bd1-9348-04e6c6efc792'), HumanMessage(content='(Now the user has been silent in a while, ask them if they are doing well.)', id='b0659798-e6bb-45ce-9994-a6693d3cf6c7'), HumanMessage(content='Hi', id='493f810b-1e05-4b3f-a026-f07670482c4e')], 'trigger': False, 'initialized': True, 'current_stage_idx': 0, 'events': [EventDescriptor(name=<CodingEventType.CODE_EDITOR_CONTENT_CHANGED: 'code_editor_content_changed'>, description='The candidate has changed the code in the editor (shown in diff format).'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_SET_UPDATED: 'user_defined_test_cases_changed'>, description='The candidate has updated the user-defined test cases set.'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_EXECUTED: 'user_defined_test_case_

In [15]:
dumped_state = dumpd(state)
dumped_state

[{'messages': [{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
    'kwargs': {'content': 'Hi',
     'type': 'human',
     'id': '2ca845d0-2987-4ccb-b977-1b85c5d27a33'}},
   {'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
    'kwargs': {'content': '(User just joined the call, please welcome and introduce yourself:)',
     'type': 'human',
     'id': '1f2f3caa-381d-4bd1-9348-04e6c6efc792'}},
   {'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
    'kwargs': {'content': '(Now the user has been silent in a while, ask them if they are doing well.)',
     'type': 'human',
     'id': 'b0659798-e6bb-45ce-9994-a6693d3cf6c7'}},
   {'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
    'kwargs': {'content': 'Hi',
     'type': 'human',
     'id': '493f810b-1e05-4b3f-a026-f07670482c4e'}}],
  'trigge

In [18]:
loaded_state = load(dumped_state, valid_namespaces=["agent_graph"])
loaded_state_snapshot = StateSnapshot(*loaded_state)
loaded_state_snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi', id='2ca845d0-2987-4ccb-b977-1b85c5d27a33'), HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='1f2f3caa-381d-4bd1-9348-04e6c6efc792'), HumanMessage(content='(Now the user has been silent in a while, ask them if they are doing well.)', id='b0659798-e6bb-45ce-9994-a6693d3cf6c7'), HumanMessage(content='Hi', id='493f810b-1e05-4b3f-a026-f07670482c4e')], 'trigger': False, 'initialized': True, 'current_stage_idx': 0, 'events': [EventDescriptor(name=<CodingEventType.CODE_EDITOR_CONTENT_CHANGED: 'code_editor_content_changed'>, description='The candidate has changed the code in the editor (shown in diff format).'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_SET_UPDATED: 'user_defined_test_cases_changed'>, description='The candidate has updated the user-defined test cases set.'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_EXECUTED: 'user_defined_test_case_

In [19]:
loaded_state_snapshot == state, dumpd(loaded_state_snapshot) == dumpd(state)

(False, True)

In [20]:
loaded_state_snapshot.values == state.values

True

In [10]:
history = list(graph.get_state_history(config=config))

In [11]:
dumped_history = dumps(history)
dumped_history

'[[{"messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "Hi", "type": "human", "id": "2ca845d0-2987-4ccb-b977-1b85c5d27a33"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "(User just joined the call, please welcome and introduce yourself:)", "type": "human", "id": "1f2f3caa-381d-4bd1-9348-04e6c6efc792"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "(Now the user has been silent in a while, ask them if they are doing well.)", "type": "human", "id": "b0659798-e6bb-45ce-9994-a6693d3cf6c7"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "Hi", "type": "human", "id": "493f810b-1e05-4b3f-a026-f07670482c4e"}}], "trigger": false, "initialized": true, "current_stage_idx": 0, "events": [{"lc": 1, "type": "constructor

In [12]:
loaded_history = loads(dumped_history, valid_namespaces=["agent_graph"])
loaded_history_snapshot = [StateSnapshot(*h) for h in loaded_history]
loaded_history_snapshot

[StateSnapshot(values={'messages': [HumanMessage(content='Hi', id='2ca845d0-2987-4ccb-b977-1b85c5d27a33'), HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='1f2f3caa-381d-4bd1-9348-04e6c6efc792'), HumanMessage(content='(Now the user has been silent in a while, ask them if they are doing well.)', id='b0659798-e6bb-45ce-9994-a6693d3cf6c7'), HumanMessage(content='Hi', id='493f810b-1e05-4b3f-a026-f07670482c4e')], 'trigger': False, 'initialized': True, 'current_stage_idx': 0, 'events': [EventDescriptor(name=<CodingEventType.CODE_EDITOR_CONTENT_CHANGED: 'code_editor_content_changed'>, description='The candidate has changed the code in the editor (shown in diff format).'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_SET_UPDATED: 'user_defined_test_cases_changed'>, description='The candidate has updated the user-defined test cases set.'), EventDescriptor(name=<CodingEventType.USER_DEFINED_TEST_CASE_EXECUTED: 'user_defined_test_case

In [13]:
loaded_history_snapshot == history, dumpd(loaded_history_snapshot) == dumpd(history)

(False, True)

In [14]:
dumped_state_byte = dumped_state.encode()
dumped_history_byte = dumped_history.encode()
print(f"State: {len(dumped_state_byte) / 1024:.2f} KB")
print(f"State History: {len(dumped_history_byte) / 1024:.2f} KB")

State: 2.26 KB
State History: 14.24 KB


In [ ]:
!pip install graphviz

In [ ]:
a = graph.get_graph().draw_mermaid()

In [7]:
with open("graph.md", "w") as f:
    f.write(a)

In [ ]:
from pydantic.v1 import BaseModel, Field, PrivateAttr


class User(BaseModel):
    name: str
    age: int
    _a: str = PrivateAttr(...)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        print("base class init")
        self._a = "a"


class User2(User):

    c: str = Field(...)


user = User2(name="Charlie", age=20, c="c")
print(user)  # Output: 50

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

CACHE_CONFIG = {"cache_control": {"type": "ephemeral"}}

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            template="You are a {role} assistant." * 1000,
            additional_kwargs=CACHE_CONFIG,
        ),
        HumanMessagePromptTemplate.from_template(
            template="{input}",
            additional_kwargs=CACHE_CONFIG,
        ),
    ]
)

chat = ChatAnthropic(
    model_name="claude-3-5-haiku-20241022",
    extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"},  # type: ignore
)

openai = ChatOpenAI(name="gpt-4o-mini")

chain = prompt | chat
# chain.invoke(
#     {
#         "role": "helpful",
#         "input": "What is second page of the shakespeare?",
#     }
# ).dict()


In [ ]:
openai.invoke(
    [
        SystemMessage(
            content=[
                {
                    "text": "You are a helpful assistant." * 1000,
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
        HumanMessage(
            content=[
                {
                    "text": "What is second page of the shakespeare?",
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
        HumanMessage(
            content=[
                {
                    "text": "What is first page of the shakespeare?",
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
    ]
).dict()

In [ ]:
import json
import time


json.dumps(
    {
        "context": "test",
        "name": "test",
        "time": time.time(),
    }
)
